In [ ]:
import tensorflow as tf
import numpy as np
import os
import pickle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from string import punctuation
from google.colab import drive
import string

In [ ]:
import requests
data = requests.get('http://www.gutenberg.org/cache/epub/11/pg11.txt').text


In [ ]:


if not os.path.isdir("data"):
    os.mkdir("data")
open("data/wonderland.txt", "w", encoding="utf-8").write(data)

167899

In [ ]:
print(data)

﻿
    The Project Gutenberg eBook of Alice's Adventures in Wonderland
    
This ebook is for the use of anyone anywhere in the United States and 
most other parts of the world at no cost and with almost no restrictions 
whatsoever. You may copy it, give it away or re-use it under the terms 
of the Project Gutenberg License included with this ebook or online 
at www.gutenberg.org. If you are not located in the United States, 
you will have to check the laws of the country where you are located 
before using this eBook.



    
        Title: Alice's Adventures in Wonderland
        
        Author: Lewis Carroll

        
        Release date: June 27, 2008 [eBook #11]Most recently updated: March 30, 2021
        Language: English
        
        Credits: Arthur DiBianca and David Widger
    
        
            *** START OF THE PROJECT GUTENBERG EBOOK ALICE'S ADVENTURES IN WONDERLAND ***
        
[Illustration]




Alice’s Adventures in Wonderland

by Lewis Carroll

THE MILLENNIUM FU

In [ ]:
len(data)

167899

In [ ]:
# content = data[960:].split('\n')
# content[0]

In [ ]:
with open("data/wonderland.txt","w",encoding="utf-8") as f:
  f.write(data)

In [ ]:
sequence_length = 100
BATCH_SIZE = 128
EPOCHS = 30
# dataset file path
FILE_PATH = "data/wonderland.txt"
BASENAME = os.path.basename(FILE_PATH)
# read the data
text = open(FILE_PATH, encoding="utf-8").read()
# remove caps, comment this code if you want uppercase characters as well
text = text.lower()
# remove punctuation
text = text.translate(str.maketrans("", "", punctuation))

In [ ]:
print(text)

﻿
    the project gutenberg ebook of alices adventures in wonderland
    
this ebook is for the use of anyone anywhere in the united states and 
most other parts of the world at no cost and with almost no restrictions 
whatsoever you may copy it give it away or reuse it under the terms 
of the project gutenberg license included with this ebook or online 
at wwwgutenbergorg if you are not located in the united states 
you will have to check the laws of the country where you are located 
before using this ebook



    
        title alices adventures in wonderland
        
        author lewis carroll

        
        release date june 27 2008 ebook 11most recently updated march 30 2021
        language english
        
        credits arthur dibianca and david widger
    
        
             start of the project gutenberg ebook alices adventures in wonderland 
        
illustration




alice’s adventures in wonderland

by lewis carroll

the millennium fulcrum edition 30

contents

 c

In [ ]:
def remove_non_ascii(text):
    return ''.join(filter(lambda x: x in string.ascii_lowercase+"0123456789"+" ", text))

text = remove_non_ascii(text)

In [ ]:
text

'    the project gutenberg ebook of alices adventures in wonderland    this ebook is for the use of anyone anywhere in the united states and most other parts of the world at no cost and with almost no restrictions whatsoever you may copy it give it away or reuse it under the terms of the project gutenberg license included with this ebook or online at wwwgutenbergorg if you are not located in the united states you will have to check the laws of the country where you are located before using this ebook            title alices adventures in wonderland                author lewis carroll                release date june 27 2008 ebook 11most recently updated march 30 2021        language english                credits arthur dibianca and david widger                         start of the project gutenberg ebook alices adventures in wonderland         illustrationalices adventures in wonderlandby lewis carrollthe millennium fulcrum edition 30contents chapter i     down the rabbithole chapter 

In [ ]:
# print some stats
n_chars = len(text)
vocab = ''.join(sorted(set(text)))
print("unique_chars:", vocab)
n_unique_chars = len(vocab)
print("Number of characters:", n_chars)
print("Number of unique characters:", n_unique_chars)

unique_chars:  0123456789abcdefghijklmnopqrstuvwxyz
Number of characters: 151221
Number of unique characters: 37


In [ ]:
# dictionary that converts characters to integers
char2int = {}
for i,c in enumerate(vocab):
  char2int[c] = i
int2char = {}
for i,c in enumerate(vocab):
  int2char[i] = c

In [ ]:
print(char2int)

{' ': 0, '0': 1, '1': 2, '2': 3, '3': 4, '4': 5, '5': 6, '6': 7, '7': 8, '8': 9, '9': 10, 'a': 11, 'b': 12, 'c': 13, 'd': 14, 'e': 15, 'f': 16, 'g': 17, 'h': 18, 'i': 19, 'j': 20, 'k': 21, 'l': 22, 'm': 23, 'n': 24, 'o': 25, 'p': 26, 'q': 27, 'r': 28, 's': 29, 't': 30, 'u': 31, 'v': 32, 'w': 33, 'x': 34, 'y': 35, 'z': 36}


In [ ]:
# save these dictionaries for later generation
pickle.dump(char2int, open(f"{BASENAME}-char2int.pickle", "wb"))
pickle.dump(int2char, open(f"{BASENAME}-int2char.pickle", "wb"))

In [ ]:
encoded_text = np.array([char2int[c] for c in text])

In [ ]:
len(encoded_text)

151221

In [ ]:
encoded_text

array([ 0,  0,  0, ..., 25, 21, 29])

In [ ]:
text

'    the project gutenberg ebook of alices adventures in wonderland    this ebook is for the use of anyone anywhere in the united states and most other parts of the world at no cost and with almost no restrictions whatsoever you may copy it give it away or reuse it under the terms of the project gutenberg license included with this ebook or online at wwwgutenbergorg if you are not located in the united states you will have to check the laws of the country where you are located before using this ebook            title alices adventures in wonderland                author lewis carroll                release date june 27 2008 ebook 11most recently updated march 30 2021        language english                credits arthur dibianca and david widger                         start of the project gutenberg ebook alices adventures in wonderland         illustrationalices adventures in wonderlandby lewis carrollthe millennium fulcrum edition 30contents chapter i     down the rabbithole chapter 

In [ ]:
# construct tf.data.Dataset object
char_dataset = tf.data.Dataset.from_tensor_slices(encoded_text)

In [ ]:
for char in char_dataset.take(5):
    print(char.numpy(), int2char[char.numpy()])

0  
0  
0  
0  
30 t


In [ ]:
# build sequences by batching
sequences = char_dataset.batch(2*sequence_length + 1, drop_remainder=True)
#The drop_remainder parameter is set to True. This means that if the dataset does not contain enough data to create a full batch, the batch will be dropped.
# print sequences
for sequence in sequences.take(2):
    print(''.join([int2char[i] for i in sequence.numpy()]))

    the project gutenberg ebook of alices adventures in wonderland    this ebook is for the use of anyone anywhere in the united states and most other parts of the world at no cost and with almost no r
estrictions whatsoever you may copy it give it away or reuse it under the terms of the project gutenberg license included with this ebook or online at wwwgutenbergorg if you are not located in the unit


In [ ]:
def split_sample(sample):
    # example :
    # sequence_length is 10
    # sample is "python is a great pro" (21 length)
    # ds will equal to ('python is ', 'a') encoded as integers
    ds = tf.data.Dataset.from_tensors((sample[:sequence_length], sample[sequence_length]))
    for i in range(1, (len(sample)-1) // 2):
        # first (input_, target) will be ('ython is a', ' ')
        # second (input_, target) will be ('thon is a ', 'g')
        # third (input_, target) will be ('hon is a g', 'r')
        # and so on
        input_ = sample[i: i+sequence_length]
        target = sample[i+sequence_length]
        # extend the dataset with these samples by concatenate() method
        other_ds = tf.data.Dataset.from_tensors((input_, target))
        ds = ds.concatenate(other_ds)
    return ds

# prepare inputs and targets
dataset = sequences.flat_map(split_sample)

In [ ]:
def one_hot_samples(input_, target):
    # onehot encode the inputs and the targets
    # Example:
    # if character 'd' is encoded as 3 and n_unique_chars = 5
    # result should be the vector: [0, 0, 0, 1, 0], since 'd' is the 4th character
    return tf.one_hot(input_, n_unique_chars), tf.one_hot(target, n_unique_chars)

dataset = dataset.map(one_hot_samples)

In [ ]:
# print first 2 samples
for element in dataset.take(1):
    print("Input:", ''.join([int2char[np.argmax(char_vector)] for char_vector in element[0].numpy()]))
    print("Target:", int2char[np.argmax(element[1].numpy())])
    print("Input shape:", element[0].shape)
    print("Target shape:", element[1].shape)
    print("="*50, "\n")

Input:     the project gutenberg ebook of alices adventures in wonderland    this ebook is for the use of a
Target: n
Input shape: (100, 37)
Target shape: (37,)



In [ ]:
# repeat, shuffle and batch the dataset
ds = dataset.repeat().shuffle(1024).batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
model = Sequential([
    LSTM(256, input_shape=(sequence_length, n_unique_chars), return_sequences=True),
    Dropout(0.3),
    LSTM(256),
    Dense(n_unique_chars, activation="softmax"),
])

In [ ]:
# define the model path
model_weights_path = f"results/{BASENAME}-{sequence_length}.h5"
model.summary()
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100, 256)          301056    
                                                                 
 dropout (Dropout)           (None, 100, 256)          0         
                                                                 
 lstm_1 (LSTM)               (None, 256)               525312    
                                                                 
 dense (Dense)               (None, 37)                9509      
                                                                 
Total params: 835,877
Trainable params: 835,877
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# make results folder if does not exist yet
if not os.path.isdir("results"):
    os.mkdir("results")
# train the model
model.fit(ds, steps_per_epoch=(len(encoded_text) - sequence_length) // BATCH_SIZE, epochs=EPOCHS)
# save the model
model.save(model_weights_path)

Epoch 1/30
1180/1180 [==============================] - 51s 34ms/step - loss: 2.3407 - accuracy: 0.3226
Epoch 2/30
1180/1180 [==============================] - 40s 34ms/step - loss: 1.8490 - accuracy: 0.4571
Epoch 3/30
1180/1180 [==============================] - 40s 34ms/step - loss: 1.6357 - accuracy: 0.5155
Epoch 4/30
1180/1180 [==============================] - 39s 33ms/step - loss: 1.4840 - accuracy: 0.5571
Epoch 5/30
1180/1180 [==============================] - 39s 33ms/step - loss: 1.3511 - accuracy: 0.5940
Epoch 6/30
1180/1180 [==============================] - 40s 34ms/step - loss: 1.2311 - accuracy: 0.6274
Epoch 7/30
1180/1180 [==============================] - 39s 33ms/step - loss: 1.1181 - accuracy: 0.6594
Epoch 8/30
1180/1180 [==============================] - 39s 33ms/step - loss: 1.0153 - accuracy: 0.6877
Epoch 9/30
1180/1180 [==============================] - 40s 34ms/step - loss: 0.9229 - accuracy: 0.7144
Epoch 10/30
1180/1180 [==============================] - 39s 33m

In [ ]:
import numpy as np
import pickle
import tqdm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Activation
import os

sequence_length = 100
dataset file path
FILE_PATH = "data/wonderland.txt"
FILE_PATH = "data/python_code.py"
BASENAME = os.path.basename(FILE_PATH)

SyntaxError: ignored

In [ ]:
seed = "in another moment down"

In [ ]:
 # load vocab dictionaries
char2int = pickle.load(open(f"{BASENAME}-char2int.pickle", "rb"))
int2char = pickle.load(open(f"{BASENAME}-int2char.pickle", "rb"))
vocab_size = len(char2int)

In [ ]:
# building the model
model = Sequential([
    LSTM(256, input_shape=(sequence_length, vocab_size), return_sequences=True),
    Dropout(0.3),
    LSTM(256),
    Dense(vocab_size, activation="softmax"),
])

In [ ]:
# load the optimal weights
model.load_weights(f"results/{BASENAME}-{sequence_length}.h5")

In [ ]:
s = seed
n_chars = 400
# generate 400 characters
generated = ""
for i in tqdm.tqdm(range(n_chars), "Generating text"):
    # make the input sequence
    X = np.zeros((1, sequence_length, vocab_size))
    for t, char in enumerate(seed):
        X[0, (sequence_length - len(seed)) + t, char2int[char]] = 1
    # predict the next character
    predicted = model.predict(X, verbose=0)[0]
    # converting the vector to an integer
    next_index = np.argmax(predicted)
    # converting the integer to a character
    next_char = int2char[next_index]
    # add the character to results
    generated += next_char
    # shift seed and the predicted character
    seed = seed[1:] + next_char

print("Seed:", s)
print("Generated text:")
print(generated)